# Model I/O - Prompt

## PromptTemplate 字符串提示模版

字符串提示模版最终形成的提示是以字符串形式存在的，在字符串模版中可以有多个占位符，每个占位符就是一个变量，后续可以将变量的位置替换成对应的字符串。
格式化模版的时候，可以使用 f-strings（默认）或 jinja2 语法。

引入类

In [ ]:
from langchain import PromptTemplate

### 1、使用构造函数创建

In [ ]:

# 使用构造函数创建
template = """\
秋水札记，定位于 {product1} AI 编程推广，分享 {product2} 商业与技术。
"""
prompt = PromptTemplate(input_variables=["product1","product2"], template=template)
print(prompt.format(product1="LangChain", product2="AI"))
print(prompt.format_prompt(product1="LangChain", product2="AI").to_messages())

### 2、使用from_template方法，模版字符串创建

In [ ]:

# 使用from_template方法创建
template = """\
秋水札记，定位于 {product1} AI 编程推广，分享 {product2} 商业与技术。
"""
prompt_multiple = PromptTemplate.from_template(template)
print(prompt_multiple.format(product1="LangChain", product2="AI"))

### 3、使用from_file方法创建，模版文件创建

In [ ]:
# 假设有一个名为"model_io_prompt.txt"的文件，其中包含模板字符串
# 文件内容: "秋水札记，定位于 {product1} AI 编程推广，分享 {product2} 商业与技术。"
template_path = "model_io_prompt.txt"
prompt_from_file = PromptTemplate.from_file(template_path)
print(prompt_from_file.format(product1="LangChain", product2="AI"))

### 4、partial方法多次对 prompt 提示模版进行赋值

#### partial方法字符串赋值

In [ ]:
# 初始化模板，其中包含了两个变量：product1和product2
template_str = "秋水札记，定位于 {product1} AI 编程推广，分享 {product2} 商业与技术。"
prompt = PromptTemplate.from_template(template_str)

# 对模板进行partial处理，先填充product1变量
partial_prompt = prompt.partial(product1="langchain")
print(partial_prompt)

# 使用partial处理后的模板，赋值剩余的product2变量
# 最后一步赋值操作需要用format方法
final_statement = partial_prompt.format(product2="AI")
print(final_statement)

#### partial方法函数赋值

In [ ]:


def get_product_name():
    # 这里的逻辑可以根据实际情况动态决定product2的值
    # 为了示例，我们直接返回"AI"
    return "AI"

# 更新模板字符串以包含三个参数，并预先为product3赋值
prompt = PromptTemplate(
    template="{product3}，定位于 {product1} AI 编程推广，分享 {product2} 商业与技术。",
    input_variables=["product1", "product2"],
    partial_variables={"product3": "秋水札记"}  # 在构造方法中预先为product3赋值
)

# 使用函数Partial处理，这里我们假设product1始终为"LangChain"
partial_prompt = prompt.partial(product1="LangChain", product2=get_product_name)

# 格式化时，我们不需要再传递任何变量，因为它们已通过Partial处理被填充
final_statement = partial_prompt.format()
print(final_statement)

## ChatPromptTemplate 聊天提示模版

引入类

In [ ]:
from langchain.prompts import ChatPromptTemplate,AIMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain import PromptTemplate

### 1、使用构造方法创建

使用SystemMessage、HumanMessage、AIMessage通过具体提示文本创建

In [ ]:
# SystemMessage、HumanMessage、AIMessage
# 这种类型组成的message，content赋值只能是字符串，不能是模版
messages =   [
    SystemMessage(content='秋水札记，定位于 LangChain AI 编程推广，分享 LangChain 商业与技术。'),
    HumanMessage(content='帮我列出几个AI商业案例？')
 ]

#构造方法创建提示模版
prompt = ChatPromptTemplate(messages=messages)

# 格式化模板
formatted_prompt_message = prompt.format_messages()
print("format_messages()执行后返回List[BaseMessage] 对象：")
print(formatted_prompt_message)

print("\n每一个BaseMessage 对象：")
for msg in formatted_prompt_message:
    print(msg.content)

formatted_prompt = prompt.format()
print("\nformat()执行后返回字符串：")  
print(formatted_prompt)    

formatted_format_prompt = prompt.format_prompt()
print("\nformat_prompt()执行后返回PromptValue 对象：")   
print(formatted_format_prompt)   


使用SystemMessagePromptTemplate、HumanMessagePromptTemplate、AIMessagePromptTemplate通过提示模版创建

In [ ]:
# 定义变量
brand = "秋水札记"
product_promotion = "LangChain"
business_case = "AI商业案例"

# 使用构造函数创建提示模板
# 这里的 messages 列表直接使用了字符串格式的消息，
# 但同样可以根据需要扩展为更复杂的消息提示模板
messages =   [
    SystemMessagePromptTemplate(
        prompt = PromptTemplate(input_variables=[], 
                                template='{brand}，定位于 LangChain AI 编程推广，分享 {product_promotion} 商业与技术。')
    ),HumanMessagePromptTemplate(
        prompt = PromptTemplate(input_variables=[], 
                                template='帮我列出几个{business_case}')
    )
]

prompt = ChatPromptTemplate(input_variables=[], messages=messages)

# 格式化并输出模板
formatted_prompt = prompt.format_messages(brand=brand, product_promotion=product_promotion, business_case=business_case)
print("format_messages()执行后返回List[BaseMessage] 对象：")
print(formatted_prompt)


### 2、使用from_messages创建，使用包含元组（tuple）的列表（list）结构创建模版

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "{brand}，定位于 LangChain AI 编程推广，分享 {product_promotion} 商业与技术。"),
    HumanMessagePromptTemplate(
        prompt = PromptTemplate(input_variables=[], 
                                template='帮我列出几个{business_case}')
    ),
    AIMessage(content='语音助手、聊天机器人和对话式AI。')
])

messages = template.format_messages(
    brand="秋水札记",
    product_promotion="LangChain",
    business_case="AI商业案例"
)

print(messages)


## FewShotPromptTemplate 少样本提示模版

引入类

In [ ]:
from langchain import PromptTemplate
from langchain.prompts import FewShotPromptTemplate

### 1、创建一个少样例提示

In [ ]:
# 定义示例
examples = [
    {"title": "今日份的小确幸", "content": "早上的一杯咖啡，让忙碌的生活暂时按下暂停键。在这个小小的瞬间，找到了自己的小确幸。"},
    {"title": "一个人的西藏", "content": "独自一人踏上西藏之旅，每一步都是风景，每一刻都是故事。这里的蓝天白云，让我忘记了世界的喧嚣。"},
    {"title": "夏日防晒必备良品", "content": "分享我这个夏天最爱的防晒霜，轻薄不油腻，让我在炎炎夏日也能享受阳光而不畏惧。"}
]

example_prompt = PromptTemplate(
    input_variables=["title", "content"], template="标题: {title}\n内容：{content}"
)

# print(example_prompt.format(**examples[0]))

# 创建 FewShotPromptTemplate
fewShotprompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    example_separator="\n\n",
    prefix="想要创作出具有小红书风格的内容，请参考以下示例：",
    suffix="根据上述示例，尝试创作一个{user_input}。",
    input_variables=["user_input"]
)

# 生成提示
print(fewShotprompt.format(user_input="新的故事"))